In [1]:
from deepsudoku import REPO_PATH, DIFFICULTIES
import os
os.chdir(REPO_PATH)

import tensorflow as tf
import numpy as np
from deepsudoku.reinforcement_learning.ppo import *

2023-08-21 13:08:32.883146: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Sudoku Environment avaible at gymnasium as 'Sudoku-v0'.


In [2]:
envs = PPO_MultiDiscrete_Environment_Wrapper('Sudoku-v0', 5)

sudoku_input_shape = (9,9)

In [3]:
# Create models

# Implementation Detail 2: Orthogonal Initialization of hidden weights and constant initialization of biases and output weights
# Biases seem to be 0 by default in Keras
hidden_ortho_init = lambda: tf.keras.initializers.Orthogonal(gain=np.sqrt(2))
pol_out_ortho_init = lambda: tf.keras.initializers.Orthogonal(gain=0.01)
val_out_ortho_init = lambda: tf.keras.initializers.Orthogonal(gain=1)

def create_policy_model():
    inputs = tf.keras.Input(shape=sudoku_input_shape)
    x = tf.keras.layers.Flatten()(inputs)
    x = tf.keras.layers.Dense(128, activation="tanh", kernel_initializer=hidden_ortho_init())(x)
    x = tf.keras.layers.Dense(128, activation="tanh", kernel_initializer=hidden_ortho_init())(x)
    x = [tf.keras.layers.Dense(9, activation="log_softmax", kernel_initializer=pol_out_ortho_init())(x) for i in range(3)]
    x = tf.keras.layers.Concatenate()(x)
    outputs = tf.keras.layers.Reshape((3,9))(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="pi")
    return model

def create_value_function_model():
    inputs = tf.keras.Input(shape=sudoku_input_shape)
    x = tf.keras.layers.Flatten()(inputs)
    x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer=hidden_ortho_init())(x)
    #x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer=hidden_ortho_init())(x)
    outputs = tf.keras.layers.Dense(1, activation="linear", kernel_initializer=val_out_ortho_init())(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="V")
    return model


In [4]:
pi, V = PPO(envs, create_policy_model(), create_value_function_model(), TRAIN_EPOCHS=4)

2023-08-21 13:08:34.939552: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


epoch:  0  ; KL:  0  ; LR:  0.005
Collection
[5 2 8]
(250, 3, 9)
tf.Tensor(
[[-2.2057657 -2.2072337 -2.188837  -2.1976442 -2.19536   -2.1888216
  -2.1902127 -2.1956084 -2.2057595]
 [-2.1906242 -2.2053595 -2.2000947 -2.1983674 -2.1998959 -2.194398
  -2.1856806 -2.2003865 -2.2003577]
 [-2.2032664 -2.2063677 -2.1834867 -2.2055693 -2.1931129 -2.20138
  -2.1967013 -2.193201  -2.1921623]], shape=(3, 9), dtype=float32)


2023-08-21 13:08:35.331427: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at gather_nd_op.cc:47 : INVALID_ARGUMENT: index innermost dimension length must be <= params rank; saw: 4 vs. 3


InvalidArgumentError: {{function_node __wrapped__GatherNd_device_/job:localhost/replica:0/task:0/device:CPU:0}} index innermost dimension length must be <= params rank; saw: 4 vs. 3 [Op:GatherNd]